In [1]:
import socket
import numpy as np
import tensorflow as tf
import pickle
from matplotlib import pyplot as plt

# Funzione per la serializzazione dei dati da inviare via socket
def serialize_data(data):
    serialized = pickle.dumps(data)
    return serialized

# Funzione per la deserializzazione dei dati ricevuti via socket
def deserialize_data(serialized_data):
    data = pickle.loads(serialized_data)
    return data

# Funzione per dividere la rete neurale
def split_neural_network(network, split_layer_index):
    model_client = tf.keras.models.Sequential()
    for layer in network.layers[:split_layer_index]:
        model_client.add(layer)

    model_server = tf.keras.models.Sequential()
    for layer in network.layers[split_layer_index:]:
        model_server.add(layer)

    return model_client, model_server

# Creazione della rete neurale MobileNet
mobileNet = tf.keras.applications.MobileNet()

# Definizione dell'indice di divisione della rete neurale
split_layer_index = 5

# Divisione della rete neurale
model_client, model_server = split_neural_network(mobileNet, split_layer_index)

# Creazione del socket per la connessione con il client
server_address = ("localhost", 8078)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(server_address)
server_socket.listen(1)

print("Server in ascolto....")

# Accettazione della connessione del client
client_socket, client_address = server_socket.accept()
print("Connected al client:", client_address)

# Ricezione del risultato parziale dal client
data_size = int.from_bytes(client_socket.recv(8), byteorder='big')
serialized_output = b''
while len(serialized_output) < data_size:
    serialized_output += client_socket.recv(4096)
client_output = deserialize_data(serialized_output)


# Esecuzione dell'inferenza sul modello del server
server_output = model_server.predict(client_output)
print("Esecuzione dell'inferenza sul modello del server")
print(server_output.shape)
predicted_class = tf.math.argmax(server_output[0], axis=-1)
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
predicted_class_name = imagenet_labels[predicted_class]
print(predicted_class_name)


# Serializzazione e invio del risultato al client
serialized_result = serialize_data(predicted_class_name)
data_size = len(serialized_result)
client_socket.sendall(data_size.to_bytes(8, byteorder='big'))
client_socket.sendall(serialized_result)
print("Serializzazione e invio del risultato al client")

# Chiusura delle connessioni
client_socket.close()
server_socket.close()

Server in ascolto....
Connected al client: ('127.0.0.1', 55062)
Esecuzione dell'inferenza sul modello del server
(1, 1000)
jay
Serializzazione e invio del risultato al client
